In [2]:
import os
import sys
import time
import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import carla

In [10]:
#Configs
map_idx = 2
MAP = ['circut_spa', 'RaceTrack', 'RaceTrack2']

# Plan na notebook
* Określamy kierunki pomiędzy nimi (oryginalnymi punktami)
* Wybieramy punkty które mają być wyżej/niżej
* Wypłaszczamy je, lub wyrównujemy -> interpolacja liniowa między skrajnie wysokim i niskim
* Testujemy spawnując auto po drodze
* zapisujemy jako csv

# W programie
* Obrobione pliki program będzie zaczytywał licząc kierunki ad hoc. Wyppłaszczanie -> stopnie zamienić na radiany czy coś i wtedy wypłaszczać.


 

In [4]:
#Connecting to client
client = carla.Client('localhost', 2000)
client.set_timeout(5.0) # seconds

In [11]:
# Formuła ze złota
def direction_lookup(pointA, pointB):
    sin_alpha = pointB[1] - pointA[1]
    cos_alpha = pointB[0] - pointA[0]
    
    alpha = np.degrees(np.arctan2(sin_alpha, cos_alpha)) 
    
    return alpha

In [12]:
world = client.load_world(MAP[map_idx])
blueprint_library = world.get_blueprint_library()
vehicle = blueprint_library.filter('*aud*')[0]
spawn_points = world.get_map().get_spawn_points()
spectator = world.get_spectator()

if MAP[map_idx] is 'circut_spa':
    spawn_points.pop(82)
elif MAP[map_idx] is 'RaceTrack2':
    spawn_points.pop(153)
elif MAP[map_idx] is 'RaceTrack':
    spawn_points.pop(183)

In [13]:
yaws = []
for idx, sp in enumerate(spawn_points[1:]):
    yaw = direction_lookup((spawn_points[idx].location.x, spawn_points[idx].location.y),
                           (sp.location.x, sp.location.y))
    yaws.append(yaw)
    
yaw = direction_lookup((spawn_points[-1].location.x, spawn_points[-1].location.y),
                       (spawn_points[0].location.x, spawn_points[0].location.y))
yaws.append(yaw)
    
len(yaws)

153

# Vehicle check

In [14]:
z = []

# i = np.random.randint(0, high=len(spawn_points))
# i = 150

# for idx, (spawn, yaw) in enumerate(zip(spawn_points[i:i+6], yaws[i:i+6])):
for idx, (spawn, yaw) in enumerate(zip(spawn_points, yaws)):
    location = spawn.location
    rotation = carla.Rotation(spawn.rotation.pitch, float(yaw), spawn.rotation.roll)
    spawn = carla.Transform(location, rotation)
    actor = world.spawn_actor(vehicle, spawn)
    actor.apply_control(carla.VehicleControl(brake=1, hand_brake=True))
    time.sleep(1.5)
    actor_z = actor.get_location().z
    print(f'Vehicle {idx}, {yaw}, {actor_z+.5}')
    z.append(actor_z+0.5)

    if idx % 3 == 0:
        spec_loc = carla.Location(spawn.location.x, spawn.location.y, spawn.location.z+6)
        spec_rot = carla.Rotation(spawn.rotation.pitch-7., float(yaw), 0.)
        spectator.set_transform(carla.Transform(spec_loc, spec_rot))


    actor.destroy()

Vehicle 0, 24.443956042559144, 0.6866202503442764
Vehicle 1, 29.831097246154602, 0.7910591065883636
Vehicle 2, 38.53020180262353, 0.8593228161334991
Vehicle 3, 55.11392899810901, 0.9621569812297821
Vehicle 4, 75.0973595665287, 1.082294762134552
Vehicle 5, 89.50609024778383, 1.1575037837028503
Vehicle 6, 105.52410383945733, 1.238535463809967
Vehicle 7, 112.43300223865306, 1.2189466953277588
Vehicle 8, 121.75943708823901, 1.1537968516349792
Vehicle 9, 134.53421576413763, 1.1035924553871155
Vehicle 10, 157.05424516899942, 1.0479546785354614
Vehicle 11, 163.7651056266783, 0.8905927836894989
Vehicle 12, 171.20737518148323, 0.9030797183513641
Vehicle 13, 178.11594974528353, 0.9057003259658813
Vehicle 14, -173.14636582434431, 0.892123818397522
Vehicle 15, -159.20098628313983, 0.8872373998165131
Vehicle 16, -139.81351963221965, 0.8744514286518097
Vehicle 17, -115.07442507687595, 0.8616016805171967
Vehicle 18, -93.53955130063231, 0.8695107102394104
Vehicle 19, -80.24757920473019, 0.883341223001

In [15]:
spawn_points_df = pd.DataFrame().from_dict({idx: (sp.location.x, sp.location.y, sp.location.z)
                                            for idx, sp in enumerate(spawn_points)}).T

spawn_points_df.rename(columns={0:'x', 1:'y', 2:'z'}, inplace=True)
spawn_points_df['yaw'] = yaws
spawn_points_df['z'] = z
spawn_points_df.shape

spawn_points_df.to_csv(f'../data/spawn_points/{MAP[map_idx]}.csv', index=False)
len(spawn_points)

153